# Parties

# Different attributes 

In [ ]:
'at_fault': #OK
'case_id': #OK
'cellphone_use': # Problem: une chiée de '1', '2', '3' => deviner à quoi ça correspond ?? 
'financial_responsibility': #OK
'hazardous_materials': #OK,
'id': #OK
'movement_preceding_collision': #OK
'other_associate_factor_1': #OK,
'other_associate_factor_2': #OK,
'party_age': pd.Int16Dtype(), # Problem: interpretation age 999 => pregnant
'party_drug_physical': str, # Problem: 585062 rows with G corresponding to nothing => replace by None ? 
'party_number': #OK
'party_safety_equipment_1': #OK,
'party_safety_equipment_2': #OK,
'party_sex': #OK
'party_sobriety': #OK
'party_type': #OK
'school_bus_related': #OK
'statewide_vehicle_type': #OK
'vehicle_make': # Koi kil faut check ?
'vehicle_year':#OK,

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dtype_parties={
    'at_fault': bool, # bool or floats
    'case_id': np.uint64, # cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'cellphone_use': str, 
    'financial_responsibility': str,
    'hazardous_materials': str,
    'id': int,
    'movement_preceding_collision': str,
    'other_associate_factor_1': str,
    'other_associate_factor_2': str,
    'party_age': pd.Int16Dtype(),
    'party_drug_physical': str,
    'party_number': int,
    'party_safety_equipment_1': str,
    'party_safety_equipment_2': str,
    'party_sex': str,
    'party_sobriety': str,
    'party_type': str,
    'school_bus_related': str,
    'statewide_vehicle_type': str,
    'vehicle_make': str,
    'vehicle_year': pd.Int16Dtype(),
}#check what type to give for integers
#int does not allow nulls
#np.uint64 allows up to 64 bytes (necessary for case id)
#pd.Int16/32/64Dtype() allows nulls

In [ ]:
parties_data = pd.read_csv("CSV-2018/parties2018.csv",dtype=dtype_parties,usecols=dtype_parties.keys()) 

In [ ]:
parties_data

# Data cleaning

In [ ]:
clean_data = parties_data.copy()

Translation tables for data cleaning

In [ ]:
translation_movement_preceding_collision ={
    'stopped': "A",
    'proceeding straight': "B",
    'ran off road': "C",
    'making right turn': "D",
    'making left turn': "E",
    'making u-turn': "F",
    'backing': "G",
    'slowing/stopping': "H",
    'passing other vehicle': "I",
    'changing lanes': "J",
    'parking maneuver': "K",
    'entering traffic': "L",
    'other unsafe turning': "M",
    'crossed into opposing lane': "N",
    'parked': "O",
    'merging': "P",
    'traveling wrong way': "Q",
    'other': "R",
}

translation_party_sex ={
    'female': "F",
    'male': "M",
}

translation_party_type ={
    'driver': 1,
    'pedestrian': 2,
    'parked vehicle': 3,
    'bicyclist': 4, 
    'other': 5,
}

translation_statewide_vehicle_type ={
    'passenger car': "A",
    'passenger car with trailer':"B",
    'motorcycle or scooter': "C",
    'pickup or panel truck': "D",
    'pickup or panel truck with trailer': "E",
    'truck or truck tractor': "F",
    'truck or truck tractor with trailer': "G",
    'schoolbus': "H",
    'other bus': "I",
    'emergency vehicle': "J",
    'highway construction equipment': "K",
    'bicycle': "L",
    'other vehicle':"M",
    'pedestrian': "N",
    'moped':"O", 
}

Clean columns that need to be cleaned

In [ ]:
clean_data["movement_preceding_collision"] = parties_data["movement_preceding_collision"].map(lambda s: translation_movement_preceding_collision.get(s, None))
clean_data["party_sex"] = parties_data["party_sex"].map(lambda s: translation_party_sex.get(s, None))
clean_data["party_type"] = parties_data["party_type"].map(lambda s: translation_party_type.get(s, None)).astype(pd.Int16Dtype())
clean_data["statewide_vehicle_type"] = parties_data["statewide_vehicle_type"].map(lambda s: translation_statewide_vehicle_type.get(s, None))


# Problems with data

In [ ]:
cellphone_use_one = parties_data[parties_data["cellphone_use"] == '1']
cellphone_use_two = parties_data[parties_data["cellphone_use"] == '2']
cellphone_use_three = parties_data[parties_data["cellphone_use"] == '3']

cellphone_use_b = parties_data[parties_data["cellphone_use"] == 'B']
cellphone_use_c = parties_data[parties_data["cellphone_use"] == 'C']
cellphone_use_d = parties_data[parties_data["cellphone_use"] == 'D']

print(f"Set of values for cellphone_use: ")
set(parties_data["cellphone_use"])


In [ ]:
numbers = len(cellphone_use_one) + len(cellphone_use_two) + len(cellphone_use_three)
letters = len(cellphone_use_b) + len(cellphone_use_c) + len(cellphone_use_d)
print(f"Occurences of 1,2,3: {numbers}")
print(f"Occurences of B,C,D: {letters}")


We realised that the values that are stored in the cellphone_use column {'1', '2', '3', 'B', 'C', 'D', nan} are different to the ones on the handout {'B', 'C', 'D', nan}. The values that are in the data but not in the handout {'1', '2', '3'} appear 2636894 times. We decided not to drop these values because they are a big chunk of the data (56%). 

We needed to find a plausible mapping between the numbers and the letters. We opted to do it by doing a frequency analysis. As you can see in the next cell, it is clear that 1 and B are those that appear the least, and 3 and D are those that appear most frequently. 

Therefore, we concluded that the correct mapping is: 1 -> B, 2 -> C, 3 -> D



In [ ]:
print(f"1 : {len(cellphone_use_one)} in % : {len(cellphone_use_one)/numbers:.3f} \t\t B : {len(cellphone_use_b)} in % : {len(cellphone_use_b)/letters:.3f}")
print(f"2 : {len(cellphone_use_two)} in % : {len(cellphone_use_two)/numbers:.3f} \t\t C : {len(cellphone_use_c)} in % : {len(cellphone_use_c)/letters:.3f}")
print(f"3 : {len(cellphone_use_three)} in % : {len(cellphone_use_three)/numbers:.3f} \t D : {len(cellphone_use_d)} in % : {len(cellphone_use_d)/letters:.3f}")


In [ ]:
# Problems with vehicle_make

American motors | American Motors (AMC)
Dodg | Dodge
Grumman | Grumman Motor Home ?
Hond | Honda
Mazd | Mazda
Merceds benz | Mercedes-Benz
MISCELLANEOUS ? (is it null or not stated?)
Toyota | Toyta
White | White GMC | White Volvo